In [1]:
import numpy as np
from quant_rotor.models.support_ham import write_matrix_elements, basis_m_to_p_matrix_conversion
from quant_rotor.models.t_amplitudes_sub_class import QuantumSimulation, TensorData, SimulationParams
import quant_rotor.core.de_solve as de

# Time propagation.

Set initial conditions.

In [4]:
sites = 5
states = 5
low_states = 1
threshold = 1e-8
g = 0.1
i_method = 3
gap = False
gap_site = 3

In [5]:
t_a_i_tensor_initial = 0
t_ab_ij_tensor_initial = 0

Set secondary initial conditions.

In [ ]:
p = states
i = low_states
a = p - i
periodic = True

# Load .npy matrices directly from the package
K, V = write_matrix_elements((states-1)//2)

V = V + V.T - np.diag(np.diag(V))
V_tensor = V.reshape(p, p, p, p)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K)
v_full = basis_m_to_p_matrix_conversion(V_tensor)

v_full = v_full * g

if t_a_i_tensor_initial == 0 and  t_ab_ij_tensor_initial == 0:
    t_a_i_tensor = np.full((sites, a, i), t_a_i_tensor_initial, dtype=complex)
    t_ab_ij_tensor = np.full((sites, sites, a, a, i, i), t_ab_ij_tensor_initial, dtype=complex)
else:
    t_a_i_tensor = t_a_i_tensor_initial
    t_ab_ij_tensor = t_ab_ij_tensor_initial


#eigenvalues from h for update
epsilon = np.diag(h_full)

params = SimulationParams(
a=a,
i=i,
p=p,  # These can be the same as `a + i` or chosen independently
sites=sites,
states=states,
i_method=i_method,
gap=gap,
gap_site=gap_site,
epsilon=epsilon,
periodic=periodic
)

tensors = TensorData(
t_a_i_tensor=t_a_i_tensor,
t_ab_ij_tensor=t_ab_ij_tensor,
h_full=h_full,
v_full=v_full
)

qs = QuantumSimulation(params, tensors)

del K, V, h_full, v_full, t_a_i_tensor, t_ab_ij_tensor, V_tensor

single = np.zeros((sites, a, i), dtype = complex)
double = np.zeros((sites, sites, a, a, i ,i), dtype = complex)

In [ ]:
def postprocess_rk45_integration_results(sol,t0_stored):
        
        # Copy the time value arrays
        time = sol.t.copy()

        # initialize the arrays to store the autocorrelation function
        true_evaluated_t0 = np.zeros_like(time, dtype=complex)
         
        # only extract the values which correspond to time steps in the solution
        # since we save C(t) for all integration steps, but only some are accepted
        t_dict = {t[0]: t[1] for t in t0_stored}
        for idx, t in enumerate(sol.t):
            true_evaluated_t0[idx] = t_dict[t]

        return(time,true_evaluated_t0)

Set residuals.

In [ ]:
single[0] = qs.residual_single(0)
for y_site in range(1, sites):
    single[y_site] = single[0]
    double[0, y_site] = qs.residual_double_total(0, y_site)

Set tensors.

In [ ]:
tensors.t_a_i_tensor[0] -= qs.update_one(single[0])

for site_1 in range(1, sites):
    tensors.t_a_i_tensor[site_1] = tensors.t_a_i_tensor[0]
    tensors.t_ab_ij_tensor[0, site_1] -= qs.update_two(double[0, site_1])
    for site_2 in range(1, sites):
        tensors.t_ab_ij_tensor[site_2, (site_1 + site_2) % sites] = tensors.t_ab_ij_tensor[0, site_1]

Set energies.

In [ ]:
energy = 0

#energy calculations
for site_x in range(sites):
    energy += np.einsum("ip, pi->", qs.h_term(i, p), qs.B_term(i, site_x)) * 0.5

    for site_y in range(site_x + 1, site_x + sites):
        # noinspection SpellCheckingInspection
        energy += np.einsum("ijab, abij->", qs.v_term(i, i, a, a, site_x, site_y % sites), qs.t_term(site_x, site_y % sites)) * 0.5
        # noinspection SpellCheckingInspection
        energy += np.einsum("ijpq, pi, qj->", qs.v_term(i, i, p, p, site_x, site_y % sites), qs.B_term(i, site_x), qs.B_term(i, site_y % sites)) * 0.5


In [ ]:
def set_init_conditions(
    sites: int,
    states: int,
    low_states: int,
    t_a_i_tensor_initial: np.ndarray,
    t_ab_ij_tensor_initial: np.ndarray,
    threshold: float,
    g: float,
    i_method: int,
    gap: bool,
    gap_site: int
):

    p = states
    i = low_states
    a = p - i
    periodic = True

    # Load .npy matrices directly from the package
    K, V = write_matrix_elements((states-1)//2)

    V = V + V.T - np.diag(np.diag(V))
    V_tensor = V.reshape(p, p, p, p)  # Adjust if needed

    h_full = basis_m_to_p_matrix_conversion(K)
    v_full = basis_m_to_p_matrix_conversion(V_tensor)

    v_full = v_full * g

    if t_a_i_tensor_initial == 0 and  t_ab_ij_tensor_initial == 0:
        t_a_i_tensor = np.full((sites, a, i), t_a_i_tensor_initial, dtype=complex)
        t_ab_ij_tensor = np.full((sites, sites, a, a, i, i), t_ab_ij_tensor_initial, dtype=complex)
    else:
        t_a_i_tensor = t_a_i_tensor_initial
        t_ab_ij_tensor = t_ab_ij_tensor_initial


    #eigenvalues from h for update
    epsilon = np.diag(h_full)

    params = SimulationParams(
    a=a,
    i=i,
    p=p,  # These can be the same as `a + i` or chosen independently
    sites=sites,
    states=states,
    i_method=i_method,
    gap=gap,
    gap_site=gap_site,
    epsilon=epsilon,
    periodic=periodic
    )

    tensors = TensorData(
    t_a_i_tensor=t_a_i_tensor,
    t_ab_ij_tensor=t_ab_ij_tensor,
    h_full=h_full,
    v_full=v_full
    )

    qs = QuantumSimulation(params, tensors)

    del K, V, h_full, v_full, t_a_i_tensor, t_ab_ij_tensor, V_tensor

    return qs

In [ ]:
def tdcc_differential_equation(t:float,T_amp_flat:np.array,states: int, sites: int, g: float,t0_stored) -> np.array:
    """Set of coupled odes for a given time step numerically for the 1 electron hamiltonian_dict for the T_ai and T_0 equation for use
    in the scipy ode solver  

    Parameters
    ----------
    t : float
        Some value of time 
    T_ai_T_0_flat : np.array
        Flattened T_ai matrix concatenated with the T_0 value to be used in their respective ODE  
    H : np.array
        1 electron hamiltonian_dict 
    reference_state : int
        Number of electron to create the reference configuration used to the initial value problem of the coupled TDCC odes
        ex. reference_state = 2, has corresponding occupation number vector (1,1,0,0)
    thermofield : bool, optional
        Parameter for whether the hamiltonian_dict used is the thermofield hamiltonian_dict such that the correct partition of the hamiltonian_dict 
        is used, by default False

    Returns
    -------
    np.array
        the flattened array containing the derivative of the T_ai and T_0 for a given time step, the 2 derivatives are concatenated 
        to make a 1d array  
    """  

    qs = set_init_conditions(sites, states, 1, 0, 0, 1e-8, g, 3, False, 3)

    single = np.zeros((sites, a, i), dtype = complex)
    double = np.zeros((sites, sites, a, a, i ,i), dtype = complex)

    energy = 0

    while True:

        single[0] = qs.residual_single(0)
        for y_site in range(1, sites):
            single[y_site] = single[0]
            double[0, y_site] = qs.residual_double_total(0, y_site)

        one_max = single.flat[np.argmax(np.abs(single))]
        two_max = double.flat[np.argmax(np.abs(double))]

        #energy calculations
        for site_x in range(sites):
            energy += np.einsum("ip, pi->", qs.h_term(i, p), qs.B_term(i, site_x)) * 0.5

            for site_y in range(site_x + 1, site_x + sites):
                # noinspection SpellCheckingInspection
                energy += np.einsum("ijab, abij->", qs.v_term(i, i, a, a, site_x, site_y % sites), qs.t_term(site_x, site_y % sites)) * 0.5
                # noinspection SpellCheckingInspection
                energy += np.einsum("ijpq, pi, qj->", qs.v_term(i, i, p, p, site_x, site_y % sites), qs.B_term(i, site_x), qs.B_term(i, site_y % sites)) * 0.5

        if np.all(abs(single) <= threshold) and np.all(abs(double) <= threshold):
            break

        if abs(one_max) >= 100 or abs(two_max) >= 100:
            raise ValueError("Diveres.")
        
        

    dTaidt = 
    
    dT_0dt = [-1j*(np.trace(h_o_o)+  np.einsum('ia,ai->',h_o_v,T_ai))]
    
    
    dTaidt = dTaidt.flatten()
    dT_aiT_0_flat = np.concatenate([dTaidt,dT_0dt])
    t0_stored.append((t,T_0))

    return (dT_aiT_0_flat)